In [3]:
#import necessary libraries
import http.client
import json
import pandas as pd
import time

In [4]:
#my api_key. You will receive your own if you register on https://api.football-data.org 
Api_key = '295ebd7f84134205b84d083b5d5a26f6'

In [5]:
# create connection with https://api.football-data.org
# get competitions' id and name
connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': Api_key }
try:
    connection.request('GET', '/v2/competitions/', None, headers)
    response = json.loads(connection.getresponse().read().decode())
    p = response['competitions']
except:
    print('Source 2 unavailable or you reached the maximum request in particular time. You must not request more often than 10 calls/minute. Check the internet connection or Api key')
competitions = pd.DataFrame()
for item in p:
    competitions = competitions.append(item, ignore_index=True)
competitions = competitions[['id', 'name', 'code']]
competitions['id'] = competitions['id'].astype('int64')
competitions

,id,name,code
0,2006,WC Qualification CAF,QCAF
1,2023,Primera B Nacional,None
2,2024,Liga Profesional,ASL
3,2149,Copa Liga Profesional,None
4,2025,Supercopa Argentina,None
...,...,...,...
151,2061,V-League,None
152,2060,Welsh Premier League,None
153,2000,FIFA World Cup,WC
154,2155,WC Qualification CONCACAF,QCCF


In [6]:
# We use 5 top european soccer ligues
# Create a list of ids' of 5 top european soccer ligues
listo = []
kalisto = ['PL', 'BL1', 'SA', 'PD','FL1'] # code of 5 european soccer ligues (we can find from source 2)
df = competitions.query("code == 'PL'")
listo.append(df["id"].iloc[0])
df = competitions.query("code == 'BL1'")
listo.append(df["id"].iloc[0])
df = competitions.query("code == 'SA'")
listo.append(df["id"].iloc[0])
df = competitions.query("code == 'PD'")
listo.append(df["id"].iloc[0])
df = competitions.query("code == 'FL1'")
listo.append(df["id"].iloc[0])
print(listo)

[2021, 2002, 2019, 2014, 2015]


In [7]:
# get teams' id, name which participate in 5 top european soccer ligues
connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': Api_key }
teams = pd.DataFrame()
for i in listo:
    try:
        connection.request('GET', '/v2/competitions/' + str(i) + '/teams?season=2021', None, headers)
        response = json.loads(connection.getresponse().read().decode())
        p = response['teams']
    except:
        print('Source 2 unavailable or you reached the maximum request in particular time. You must not request more often than 10 calls/minute. Check the internet connection or Api key')
    for item in p:
        teams = teams.append(item, ignore_index=True)
teams = teams[['id', 'name']]
teams['id'] = teams['id'].astype('int64')
teams

,id,name
0,57,Arsenal FC
1,58,Aston Villa FC
2,61,Chelsea FC
3,62,Everton FC
4,64,Liverpool FC
...,...,...
93,545,FC Metz
94,546,Racing Club de Lens
95,547,Stade de Reims
96,548,AS Monaco FC


In [8]:
# Read in the data set from source 1 to retrieve necessary club names
df_source1 = pd.read_csv('../data/df_source1.csv')

In [10]:
# checking our DataFrame
df_source1

,name,position,Age,Nat.,Market value,club,Goals,Assists
0,Kylian Mbappé,Centre-Forward,22,France,£144.00m,Paris Saint-Germain,9,12
1,Erling Haaland,Centre-Forward,21,Norway,£135.00m,Borussia Dortmund,14,4
2,Harry Kane,Centre-Forward,28,England,£108.00m,Tottenham Hotspur,8,2
3,Jack Grealish,Left Winger,26,England,£90.00m,Manchester City,2,3
4,Mohamed Salah,Right Winger,29,Egypt,£90.00m,Liverpool FC,17,7
...,...,...,...,...,...,...,...,...
495,Giorgian de Arrascaeta,Attacking Midfield,27,Uruguay,£16.20m,Clube de Regatas do Flamengo,0,0
496,Ayoze Pérez,Second Striker,28,Spain,£16.20m,Leicester City,1,3
497,Alex Meret,Goalkeeper,24,Italy,£16.20m,SSC Napoli,0,0
498,Duje Caleta-Car,Centre-Back,25,Croatia,£16.20m,Olympique Marseille,0,0


In [12]:
# list of clubs from source 1
club_list = df_source1['club'].to_list()

In [13]:
# matching list of clubs from source 1 with source 2
club_df = teams[teams['name'].isin(club_list)]
club_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 93
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      35 non-null     int64 
 1   name    35 non-null     object
dtypes: int64(1), object(1)
memory usage: 840.0+ bytes


In [14]:
# we have 35 matches, for a simple analysis they are enough
# get list of clubs' id
id_clubs = club_df["id"].to_list()
print(id_clubs)

[57, 61, 62, 64, 328, 340, 346, 402, 2, 3, 4, 10, 11, 18, 19, 721, 98, 99, 100, 102, 103, 104, 109, 110, 113, 584, 586, 81, 82, 90, 94, 95, 559, 522, 545]


In [15]:
# get players' id, name which play for the clubs
# we use time.sleep due to this api source has limitation 10 calls/minute
# the data will be collected approximately 3-4 minutes
# If you do not want to wait, we can import already collected data from csv file, just uncomment the code (the cell) below
connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': Api_key }
players = pd.DataFrame()
for i in id_clubs:
    try:
        connection.request('GET', '/v2/teams/' + str(i), None, headers)
        response = json.loads(connection.getresponse().read().decode())
        p = response['squad']
    except:
        print('Source 2 unavailable or you reached the maximum request in particular time. You must not request more often than 10 calls/minute. Check the internet connection or Api key')
    for item in p:
        players = players.append(item, ignore_index=True)
    time.sleep(6)
players = players[['id', 'name']]
players['id'] = players['id'].astype('int64')
players

,id,name
0,110,Thomas Partey
1,3174,Bernd Leno
2,3244,Cédric
3,3477,Granit Xhaka
4,3755,Mohamed El Neny
...,...,...
974,137323,Lenny Lacroix
975,160442,William Mikelbrencis
976,165062,Ousmane Ba
977,170408,Kiki Kouyaté


In [18]:
# Read in the data set from source 1 to retrieve necessary player names
#players = pd.read_csv('../data/players_api.csv')

In [16]:
# export our data into csv file.
players.to_csv('../data/players_api.csv', index=False)

In [17]:
# list of players from source 1
player_list = df_source1['name'].to_list()

In [18]:
# matching list of players from source 1 with source 2
player_df = players[players['name'].isin(player_list)]
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202 entries, 0 to 978
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      202 non-null    int64 
 1   name    202 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.7+ KB


In [19]:
# we have 202 matches, for a simple analysis they are enough
# get list of players' id and list of player names
id_players = player_df["id"].to_list()
name_players = player_df["name"].to_list()

In [20]:
# we leave the players who matched
df_source2 = df_source1[df_source1['name'].isin(name_players)]
df_source2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202 entries, 1 to 497
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          202 non-null    object
 1   position      202 non-null    object
 2   Age           202 non-null    int64 
 3   Nat.          202 non-null    object
 4   Market value  202 non-null    object
 5   club          202 non-null    object
 6   Goals         202 non-null    int64 
 7   Assists       202 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 14.2+ KB


In [21]:
# we sort by name
player_df = player_df.sort_values(by=['name'])
df_source2 = df_source2.sort_values(by=['name'])

In [22]:
# we add the "id" column from source2 into source1
df_source2['id'] = player_df['id'].values

In [23]:
df_source2

,name,position,Age,Nat.,Market value,club,Goals,Assists,id
399,Aaron Ramsdale,Goalkeeper,23,England,£18.00m,Arsenal FC,0,0,5530
325,Abdoulaye Doucouré,Central Midfield,28,France,£22.50m,Everton FC,2,4,8119
494,Adam Armstrong,Centre-Forward,24,England,£16.20m,Southampton FC,2,3,4863
231,Adrien Rabiot,Central Midfield,26,France,£27.00m,Juventus FC,0,1,3368
439,Alessio Romagnoli,Centre-Back,26,Italy,£18.00m,AC Milan,1,0,1740
...,...,...,...,...,...,...,...,...,...
423,Yerry Mina,Centre-Back,27,Colombia,£18.00m,Everton FC,0,0,3728
124,Youssef En-Nesyri,Centre-Forward,24,Morocco,£36.00m,Sevilla FC,3,2,32836
471,Youssoufa Moukoko,Centre-Forward,17,Germany,£16.20m,Borussia Dortmund,3,1,150817
385,Yussuf Poulsen,Centre-Forward,27,Denmark,£18.90m,RB Leipzig,4,3,3466


In [24]:
# we get the number of wins, draws, and losts of players between 2021-08-01(start season) and 2021-11-23 (the day when we write this code)
# we use time.sleep due to this api source has limitation 10 calls/minute
# the data will be collected approximately 20 minutes
# If you do not want to wait, we can import already collected data from csv file, just uncomment the code (the cell) below
connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': Api_key }
d = {}
win_list = []
draw_list = []
lost_list = []
match_result = pd.DataFrame()
for i in df_source2['id'].values:
    try:
        connection.request('GET', '/v2/players/' + str(i) + '/matches?status=FINISHED&dateFrom=2021-08-01&dateTo=2021-11-23', None, headers )
        response = json.loads(connection.getresponse().read().decode())
        a = df_source2[(df_source2['id'] == i)]
        p = response['matches']
    except:
        print('Source 2 unavailable or you reached the maximum request in particular time. You must not request more often than 10 calls/minute. Check the internet connection or Api key')
    win = 0
    draw = 0
    lost = 0
    for t in p:
        s = t['score']
        h = t['homeTeam']
        nh = h['name']
        if nh in a['club'].values and s['winner'] == 'HOME_TEAM':
            win += 1
        elif s['winner'] == 'DRAW':
            draw += 1
        else:
            lost += 1
    win_list.append(win)
    draw_list.append(draw)
    lost_list.append(lost)
    time.sleep(6)
d['win'] = win_list
d['draw'] = draw_list
d['lost'] = lost_list
match_result = pd.DataFrame.from_dict(d)
match_result   

,win,draw,lost
0,4,3,6
1,3,2,3
2,2,5,5
3,3,1,8
4,5,2,9
...,...,...,...
197,3,2,2
198,5,3,2
199,6,0,6
200,5,4,5


In [27]:
# Read in the data set from source 1 to retrieve necessary match results of players
#match_result = pd.read_csv('../data/match_result.csv')

In [25]:
# export our data into csv file.
match_result.to_csv('../data/match_result.csv', index=False)

In [26]:
# we add 'win','draw','lost' columns into source1
df_source2[['win','draw','lost']] = match_result[['win','draw','lost']].values

In [27]:
# checking our DataFrame
df_source2

,name,position,Age,Nat.,Market value,club,Goals,Assists,id,win,draw,lost
399,Aaron Ramsdale,Goalkeeper,23,England,£18.00m,Arsenal FC,0,0,5530,4,3,6
325,Abdoulaye Doucouré,Central Midfield,28,France,£22.50m,Everton FC,2,4,8119,3,2,3
494,Adam Armstrong,Centre-Forward,24,England,£16.20m,Southampton FC,2,3,4863,2,5,5
231,Adrien Rabiot,Central Midfield,26,France,£27.00m,Juventus FC,0,1,3368,3,1,8
439,Alessio Romagnoli,Centre-Back,26,Italy,£18.00m,AC Milan,1,0,1740,5,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...
423,Yerry Mina,Centre-Back,27,Colombia,£18.00m,Everton FC,0,0,3728,3,2,2
124,Youssef En-Nesyri,Centre-Forward,24,Morocco,£36.00m,Sevilla FC,3,2,32836,5,3,2
471,Youssoufa Moukoko,Centre-Forward,17,Germany,£16.20m,Borussia Dortmund,3,1,150817,6,0,6
385,Yussuf Poulsen,Centre-Forward,27,Denmark,£18.90m,RB Leipzig,4,3,3466,5,4,5


In [28]:
# export our data into csv file.
df_source2.to_csv('../data/df_source1+source2.csv', index=False)